# 00_ETL_modelo

## 01_import_bibliotecas

In [1]:
import pandas as pd
import json
from pandas import json_normalize
import requests

## 02_import_dados

In [2]:
# import de dados da decision

vagas = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/vagas.json"
prospects = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/prospects.json"
applicants = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/applicants.json"

In [3]:
# lê os dados do github

# vagas
response_vagas = requests.get(vagas)
data_vagas = response_vagas.json()

# prospects
response_prospects = requests.get(prospects)
data_prospects = response_prospects.json()

# Aplicants
response_applicants = requests.get(applicants)
data_applicants = response_applicants.json()

## 03_tratamento_json

In [4]:
# Trata Vagas

records_vagas = [dict(id_vaga=k, **v) for k, v in data_vagas.items()]

df_vagas = pd.json_normalize(records_vagas, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_vagas = [
    'informacoes_basicas_',
    'perfil_vaga_',
    'beneficios_'
]

for prefixo in prefixos_para_remover_vagas:
    df_vagas.columns = df_vagas.columns.str.replace(f'^{prefixo}', '', regex=True)

In [16]:
# Trata Prospects

records_prospects = [dict(id_vaga=k, **v) for k, v in data_prospects.items()]

df_prospects = pd.json_normalize(
    records_prospects,
    record_path='prospects',
    meta=['id_vaga', 'titulo', 'modalidade'],
    sep='_'
)

# Remove os prefixos das colunas
prefixos_para_remover_prospects = [
'prospects_',  
]

for prefixo in prefixos_para_remover_prospects:
    df_prospects.columns = df_prospects.columns.str.replace(f'^{prefixo}', '', regex=True)
    
df_prospects.rename(columns={'codigo': 'id_candidato'}, inplace=True)

In [17]:
# Trata Aplicants

records_applicants = [dict(id_candidato=k, **v) for k, v in data_applicants.items()]

df_applicants = pd.json_normalize(records_applicants, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_applicants = [
    'infos_basicas_',
    'informacoes_pessoais_',
    'informacoes_profissionais_',
    'formacao_e_idiomas_',
    'cargo_atual_'
]

for prefixo in prefixos_para_remover_applicants:
    df_applicants.columns = df_applicants.columns.str.replace(f'^{prefixo}', '', regex=True)

## 04_juntando arquivos

In [49]:
# prospects + applicants
df_inter = df_prospects.merge(df_applicants, on="id_candidato", how="left", suffixes=('_tabelaprospect', '_tabelaapplicant'))

In [ ]:
# dados + Vagas
df_geral = df_inter.merge(df_vagas, on="id_vaga", how="left", suffixes=('', '_tabelavaga'))

In [50]:
df_inter.head()

,nome_tabelaprospect,id_candidato,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,id_vaga,titulo,modalidade,...,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,4530,CONSULTOR CONTROL M,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,4530,CONSULTOR CONTROL M,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,4533,2021-2605708-Microfocus Application Life Cycle...,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_geral.info() ##53759
# df_prospects.info() ##53759 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53759 entries, 0 to 53758
Columns: 111 entries, nome_tabelaprospect to nome_substituto
dtypes: object(111)
memory usage: 45.9+ MB


In [51]:
df_inter.columns.to_list()

['nome_tabelaprospect',
 'id_candidato',
 'situacao_candidado',
 'data_candidatura',
 'ultima_atualizacao',
 'comentario',
 'recrutador',
 'id_vaga',
 'titulo',
 'modalidade',
 'cv_pt',
 'cv_en',
 'telefone_recado',
 'telefone',
 'objetivo_profissional',
 'data_criacao',
 'inserido_por',
 'email',
 'local',
 'sabendo_de_nos_por',
 'data_atualizacao',
 'codigo_profissional',
 'nome_tabelaapplicant',
 'data_aceite',
 'nome_tabelaapplicant',
 'cpf',
 'fonte_indicacao',
 'email',
 'email_secundario',
 'data_nascimento',
 'telefone_celular',
 'telefone_recado',
 'sexo',
 'estado_civil',
 'pcd',
 'endereco',
 'skype',
 'url_linkedin',
 'facebook',
 'titulo_profissional',
 'area_atuacao',
 'conhecimentos_tecnicos',
 'certificacoes',
 'outras_certificacoes',
 'remuneracao',
 'nivel_profissional',
 'nivel_academico',
 'nivel_ingles',
 'nivel_espanhol',
 'outro_idioma',
 'instituicao_ensino_superior',
 'cursos',
 'ano_conclusao',
 'download_cv',
 'qualificacoes',
 'experiencias',
 'outro_curso',

In [47]:
df_geral.to_csv(r"C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5_old\\01_dados_anonimizados_decision\\dados.csv", encoding ="utf8" , sep=";" ,index=False)